In [6]:
# import radiomics
# import logging
# radiomics.setVerbosity(logging.DEBUG)
# import os
# os.environ['BUILD_CUDA_EXTENSIONS'] = '1'

# Set CUDA architecture flags for compute capability 6.0 or higher (Pascal, Volta, Turing, Ampere, etc.)
# os.environ['NVCC_FLAGS'] = '-gencode=arch=compute_60,code=sm_60 -gencode=arch=compute_70,code=sm_70 -gencode=arch=compute_75,code=sm_75 -gencode=arch=compute_80,code=sm_80'

In [1]:
LABEL_MAPPING = {
    1: "aorta",
    2: "left_lung",
    3: "right_lung",
    4: "trachea",
    5: "pulmonary_artery",
}
MASK_PATH = "./masks.nii.gz"
SCAN_PATH = "./scan.nii.gz"
CONFIG_PATH = "./extractor_config.json"

In [2]:

import os
import logging
os.environ['BUILD_CUDA_EXTENSIONS'] = '1'
!python3.10 -m pip install -r requirements.txt
# !python3.10 setup.py install
!python3.10 -m     pip install . --no-cache-dir
!python3.10 setup.py build_ext --inplace

from __future__ import print_function
import sys
import os
import logging
import six


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Processing /home/verscup/pyradiomics-CUDA
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyradiomics: filename=pyradiomics-3.1.1-cp310-cp310-linux_x86_64.whl size=218390 sha256=827760ebe6a564d1fd9acb36054a65d1c546e5d148ef6cdfe8c7cd07305e251c
  Stored in directory: /tmp/pip-ephem-wheel-cache-1mxg6uv2/wheels/00/8f/1f/22ad624e1c1e550a68cb0809a32f71abd715c39d520dc48157
Successfully built pyradiomics
  Attempting uninstall: pyradiomics
    Found existing installation: pyradiomics 3.1.1
    Uninstalling pyradiomics-3.1.1:
      Successfully uninstalled pyradiomics-3.1.1
BUILD_CUDA_EXTENSIONS=1 found. Attempting to include CUDA extensions.
Found nvcc at: /usr/bin/nvcc
Found CUDA library directory based on nvcc path: /usr/lib64
NVCC_F

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Jan__6_16:45:21_PST_2023
Cuda compilation tools, release 12.0, V12.0.140
Build cuda_12.0.r12.0/compiler.32267302_0


In [3]:
from radiomics.featureextractor import RadiomicsFeatureExtractor
import os
print("Current working directory:", os.getcwd())
os.chdir('radiomics/cuda/biecek_test')
print("Current working directory after change:", os.getcwd())
print("Files in current directory:", os.listdir())

Checking GPU availability...
NVIDIA GPU not found via nvidia-smi.
CUDA Toolkit (nvcc) found.
GPU Availability Check Result: False (GPU: False, CUDA Toolkit: True)
Current working directory: /home/verscup/pyradiomics-CUDA
Current working directory after change: /home/verscup/pyradiomics-CUDA/radiomics/cuda/biecek_test
Files in current directory: ['test.py', '.gitignore', 'testImportat.ipynb', 'scan.nii.gz', 'extractor_config.json', 'masks.nii.gz']


In [4]:
extractor = RadiomicsFeatureExtractor()
extractor.loadParams(CONFIG_PATH)

In [5]:
features = {}
for val, name in LABEL_MAPPING.items():
    features[name] = extractor.execute(SCAN_PATH, MASK_PATH, val)
    

Shape features successfully written to data/2025-03-30_20-32-22
Shape features successfully written to data/2025-03-30_20-33-11
Shape features successfully written to data/2025-03-30_20-34-02
Shape features successfully written to data/2025-03-30_20-34-03
Shape features successfully written to data/2025-03-30_20-34-04


In [7]:
# print(features)

{'aorta': OrderedDict([('original_shape_MeshVolume', array(275206.40106234)), ('original_shape_VoxelVolume', np.float64(275447.8190758457)), ('original_shape_SurfaceArea', array(49354.19084468)), ('original_shape_SurfaceVolumeRatio', array(0.17933518)), ('original_shape_Maximum3DDiameter', array(245.71126763)), ('original_shape_Maximum2DDiameterSlice', array(130.84456334)), ('original_shape_Maximum2DDiameterColumn', array(240.69384741)), ('original_shape_Maximum2DDiameterRow', array(243.29408152)), ('original_shape_MajorAxisLength', np.float64(224.96165368058996))])}


In [7]:
# print("Files in current directory:", os.listdir())
print("Directories in data folder:", [d for d in os.listdir('data') if os.path.isdir(os.path.join('data', d))])

Directories in data folder: ['2025-03-29_22-54-38']


In [11]:
from radiomics.cuda.helpers.shape_class_handlers import load_shape_class


d = load_shape_class('2025-03-29_23-02-27')
print(d)

dsa'
Shape features successfully loaded from data/2025-03-29_23-02-27
{'surface_area': 49354.19084467997, 'volume': 275206.4010623354, 'diameters': (130.8445633369893, 240.6938474145504, 243.29408152016222, 245.71126763298608), 'pixel_spacing': array([2.        , 0.66500002, 0.66500002]), 'mask_array': array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False,

In [9]:
# print(d)
from radiomics import  cShape, cShapeCpu

a, b, c = cShape.calculate_coefficients(d['mask_array'], d['pixel_spacing'])
print(a, b, c)


49354.19084467997 275206.4010623354 (130.8445633369893, 240.6938474145504, 243.29408152016222, 245.71126763298608)
